In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Load translated dataframe
import pandas as pd
chickens = pd.read_csv('../csv/translated/chickens_translated_en.csv')
chickens

,Title,Ingredients,Steps,FlavorProfile
0,Sweet and Sour Chicken,1/2 chicken--1 tbsp cornstarch--Water--Sauce i...,"1. Clean the chicken and marinate with salt, s...","sweet, sour, savory"
1,Bacem Chicken,1 chicken (cut into 12)--500 ml coconut water-...,1. Clean the chicken thoroughly. Blend all gro...,"savory, sweet"
2,Grilled Chicken,12 chicken pieces (thighs and drumsticks)--Gro...,1. Sauté ground spices until fragrant. Add Wor...,"savory, smoky, sweet"
3,Grilled Chicken with Rujak Spice,1/2 free-range chicken--10 garlic cloves--Pepp...,"1. Blend bird's eye chilies, curly chilies, ga...","spicy, sweet, savory"
4,Sweet Soy Grilled Chicken,1/2 kg chicken wings--3 garlic cloves--2 cm tu...,"1. Blend garlic, turmeric, and salt--2. Braise...","sweet, savory"
...,...,...,...,...
1248,Chicken and Bean Sprout Stew,1/4 chicken--1 pack bean sprouts--3 shallots--...,"1. Boil water, cook chicken until tender--2. G...","savory, umami"
1249,Spicy Basil Chicken Scream,1/2 kg pre-boiled chicken--1 bunch basil--2 ca...,1. Fry the pre-boiled chicken until cooked the...,"spicy, savory, umami"
1250,Chicken and Tofu with Green Chili,"Fresh chicken, cut into small pieces--Tofu, cu...","1. Fry chicken and tofu until golden, not too ...","spicy, savory, umami"
1251,Fried Chicken with Green Chili and Ginger......😁😁,1 Chicken (Cut into 15)--1/2 Kg Large Green Ch...,1. Wash the chicken thoroughly then drain--2. ...,"spicy, savory, umami"


In [3]:
# Inspect flavour profiles
chickens["FlavorProfile"]

0        sweet, sour, savory
1              savory, sweet
2       savory, smoky, sweet
3       spicy, sweet, savory
4              sweet, savory
                ...         
1248           savory, umami
1249    spicy, savory, umami
1250    spicy, savory, umami
1251    spicy, savory, umami
1252    savory, umami, sweet
Name: FlavorProfile, Length: 1253, dtype: object

In [4]:
# Clean whitespaces
chickens["FlavorProfile"] = chickens["FlavorProfile"].str.replace(" ", "")
chickens["FlavorProfile"]

0        sweet,sour,savory
1             savory,sweet
2       savory,smoky,sweet
3       spicy,sweet,savory
4             sweet,savory
               ...        
1248          savory,umami
1249    spicy,savory,umami
1250    spicy,savory,umami
1251    spicy,savory,umami
1252    savory,umami,sweet
Name: FlavorProfile, Length: 1253, dtype: object

In [5]:
# Inspect additional flavour profiles that was not in system_prompt
flavour_profile = chickens["FlavorProfile"]
filter_flavour_profile = flavour_profile
known_categories = ["sweet", "sour", "salty", "bitter", "umami", "savory", "smoky"]
for category in known_categories:
    filter_flavour_profile = (
        filter_flavour_profile.
        str.replace(category, "").
        str.replace(r'^[^a-zA-Z]+|[^a-zA-Z]+$|,(?=,)', '', regex=True)
    )
filter_flavour_profile = filter_flavour_profile[filter_flavour_profile.str.strip().ne("")]
filter_flavour_profile

3       spicy
5       spicy
8       spicy
9       spicy
10      spicy
        ...  
1245    spicy
1246    spicy
1249    spicy
1250    spicy
1251    spicy
Name: FlavorProfile, Length: 604, dtype: object

In [6]:
# Count values
count_filtered_flavours = filter_flavour_profile.value_counts().reset_index().sort_values("count", ascending=False)
count_filtered_flavours

,FlavorProfile,count
0,spicy,377
1,slightly,61
2,aromatic,45
3,crispy,20
4,creamy,16
5,"spicy,aromatic",14
6,"aromatic,slightlyspicy",7
7,"spicy,slightly",6
8,mild,6
9,herbal,5


In [7]:
# Load zero-shot-classification model
from transformers import pipeline
model = "facebook/bart-large-mnli"
zero_shot_classifier = pipeline(
    "zero-shot-classification",
    model=model,
    device="cpu"
)

Device set to use cpu


In [8]:
# Get the flavor profile of the  first recipe entry
first_entry_flavor_profile = chickens.loc[0]["FlavorProfile"].split(",")
first_entry_flavor_profile = first_entry_flavor_profile[0:2]
first_entry_flavor_profile

['sweet', 'sour']

In [9]:
# Get the ingredients of the first entry
first_entry_ingredients = chickens.loc[0]["Ingredients"]
first_entry_ingredients

'1/2 chicken--1 tbsp cornstarch--Water--Sauce ingredients:--1 onion (cut into 8 parts)--2 cloves garlic (minced)--1 carrot (julienned)--1 red chili (diced, seeds removed)--4 tbsp tomato sauce--Salt--Sugar--Seasoning--Pepper--1 tbsp cornstarch (dissolved)'

In [10]:
# Test for zero-shot-classification task
result = zero_shot_classifier(first_entry_ingredients, first_entry_flavor_profile)
result

{'sequence': '1/2 chicken--1 tbsp cornstarch--Water--Sauce ingredients:--1 onion (cut into 8 parts)--2 cloves garlic (minced)--1 carrot (julienned)--1 red chili (diced, seeds removed)--4 tbsp tomato sauce--Salt--Sugar--Seasoning--Pepper--1 tbsp cornstarch (dissolved)',
 'labels': ['sweet', 'sour'],
 'scores': [0.5984093546867371, 0.40159058570861816]}

In [11]:
# Get max score and most dominant flavor profile
import numpy as np
dominant_fp = result["labels"][np.argmax(result["scores"])]
dominant_fp

'sweet'

In [12]:
# Create function to predict most dominant flavor profile to the recipe
def predict_most_dominant_fp(recipe):
    classified = zero_shot_classifier(recipe["Ingredients"], recipe["FlavorProfile"])
    max_score_index = np.argmax(classified["scores"])
    return classified["labels"][max_score_index]
first_entry = chickens.loc[0]
predict_most_dominant_fp(first_entry)

'savory'

In [13]:
# Customize the progress bar appearance
from tqdm.notebook import tqdm
tqdm.pandas(
    desc="Predicting Dominant Flavor Profile",
    bar_format = "{l_bar}{bar}|{n_fmt}/{total_fmt}[{remaining}]",
    colour='blue'
)

In [14]:
# Start prediction
chickens['DominantFlavor'] = chickens.progress_apply(predict_most_dominant_fp, axis=1)
chickens

Predicting Dominant Flavor Profile:   0%|          |0/1253[?]

,Title,Ingredients,Steps,FlavorProfile,DominantFlavor
0,Sweet and Sour Chicken,1/2 chicken--1 tbsp cornstarch--Water--Sauce i...,"1. Clean the chicken and marinate with salt, s...","sweet,sour,savory",savory
1,Bacem Chicken,1 chicken (cut into 12)--500 ml coconut water-...,1. Clean the chicken thoroughly. Blend all gro...,"savory,sweet",sweet
2,Grilled Chicken,12 chicken pieces (thighs and drumsticks)--Gro...,1. Sauté ground spices until fragrant. Add Wor...,"savory,smoky,sweet",savory
3,Grilled Chicken with Rujak Spice,1/2 free-range chicken--10 garlic cloves--Pepp...,"1. Blend bird's eye chilies, curly chilies, ga...","spicy,sweet,savory",spicy
4,Sweet Soy Grilled Chicken,1/2 kg chicken wings--3 garlic cloves--2 cm tu...,"1. Blend garlic, turmeric, and salt--2. Braise...","sweet,savory",sweet
...,...,...,...,...,...
1248,Chicken and Bean Sprout Stew,1/4 chicken--1 pack bean sprouts--3 shallots--...,"1. Boil water, cook chicken until tender--2. G...","savory,umami",savory
1249,Spicy Basil Chicken Scream,1/2 kg pre-boiled chicken--1 bunch basil--2 ca...,1. Fry the pre-boiled chicken until cooked the...,"spicy,savory,umami",spicy
1250,Chicken and Tofu with Green Chili,"Fresh chicken, cut into small pieces--Tofu, cu...","1. Fry chicken and tofu until golden, not too ...","spicy,savory,umami",spicy
1251,Fried Chicken with Green Chili and Ginger......😁😁,1 Chicken (Cut into 15)--1/2 Kg Large Green Ch...,1. Wash the chicken thoroughly then drain--2. ...,"spicy,savory,umami",spicy


In [15]:
# Finalize data processing
path = "../csv/translated"
eggs = pd.read_csv(f"{path}/eggs_translated_en.csv")
fishs = pd.read_csv(f"{path}/fishs_translated_en.csv")
lambs = pd.read_csv(f"{path}/lambs_translated_en.csv")
tofus = pd.read_csv(f"{path}/tofus_translated_en.csv")
soybeans = pd.read_csv(f"{path}/soybeans_translated_en.csv")
prawns = pd.read_csv(f"{path}/prawns_translated_en.csv")

In [16]:
# Assign dominant flavor profile
df_names = ["eggs", "fishs", "lambs","tofus", "soybeans", "prawns"]
dfs = [eggs, fishs, lambs, tofus, soybeans, prawns]
print("Start Assignments ...")
for name, df in zip(df_names, dfs):
   with tqdm(
           total=len(df),
           bar_format = "{l_bar}{bar}|{n_fmt}/{total_fmt}[{remaining}]",
           colour="blue",
           desc=f"Processing {name}") as pbar:
        df["DominantFlavor"] = df.apply(
            lambda x: (pbar.update(1), predict_most_dominant_fp(x))[1],
            axis=1
        )
        df.to_csv(f"../csv/temp/final_{name}_temp.csv", index=False)
print("Assignments Completed")

Start Assignments ...


Processing eggs:   0%|          |0/1504[?]

Processing fishs:   0%|          |0/1382[?]

Processing lambs:   0%|          |0/1000[?]

Processing tofus:   0%|          |0/1263[?]

Processing soybeans:   0%|          |0/1158[?]

Processing prawns:   0%|          |0/1270[?]

Assignments Completed


In [17]:
# Combine all indonesian recipes
indonesian_recipes = pd.concat([
    chickens,
    eggs,
    fishs,
    lambs,
    soybeans,
    prawns,
    tofus
], axis=0)
indonesian_recipes

,Title,Ingredients,Steps,FlavorProfile,DominantFlavor
0,Sweet and Sour Chicken,1/2 chicken--1 tbsp cornstarch--Water--Sauce i...,"1. Clean the chicken and marinate with salt, s...","sweet,sour,savory",savory
1,Bacem Chicken,1 chicken (cut into 12)--500 ml coconut water-...,1. Clean the chicken thoroughly. Blend all gro...,"savory,sweet",sweet
2,Grilled Chicken,12 chicken pieces (thighs and drumsticks)--Gro...,1. Sauté ground spices until fragrant. Add Wor...,"savory,smoky,sweet",savory
3,Grilled Chicken with Rujak Spice,1/2 free-range chicken--10 garlic cloves--Pepp...,"1. Blend bird's eye chilies, curly chilies, ga...","spicy,sweet,savory",spicy
4,Sweet Soy Grilled Chicken,1/2 kg chicken wings--3 garlic cloves--2 cm tu...,"1. Blend garlic, turmeric, and salt--2. Braise...","sweet,savory",sweet
...,...,...,...,...,...
1258,Braised Pindang with Tofu,6 pindang fish--6 fried tofu--Spices:--4 shall...,1. Clean and fry fish--2. Slice spices--3. Sau...,"savory,umami,spicy",spicy
1259,Chicken with Tofu Sauce,250 gr chicken breast--1 egg--2 tbsp tapioca f...,"1. Mix chicken with egg, flour, salt, and pepp...","savory,umami",umami
1260,DEBM Style Tofu,4 eggs--1 triangle coconut milk--1 pack plain ...,1. Blend all ingredients--2. Steam for 20 minu...,"savory,umami",savory
1261,Simple Crispy Fried Tofu,4 white tofu (half-fried)--6 tbsp flour--3 tbs...,"1. Half-fry tofu--2. Grind spices, mix with fl...","savory,umami",savory


In [19]:
# Save final recipes
indonesian_recipes.to_csv(f"../csv/final/indonesian_recipes.csv", index=False)
print("Indonesian Recipes Completed")

Indonesian Recipes Completed
